In [30]:
from datasets import load_from_disk, load_metric
from transformers import RobertaTokenizer, RobertaForSequenceClassification, RobertaConfig, Trainer
import numpy as np
import pyarrow as pa
import pandas as pd

#import os
#import sys
#sys.path.insert(0, '/zhome/a6/6/127219/Speciale/master_project')
#from src.models.transformers_modeling_roberta import RobertaForSequenceClassification

In [31]:
# load
datadir = '/work3/s174498/sst2_dataset/'
checkpoint = "/work3/s174498/finetuning-sentiment-model-test-head3/checkpoint-500"

# test data
test_dataset = load_from_disk(datadir + 'test_dataset')

# tokenizer
tokenizer = RobertaTokenizer.from_pretrained(checkpoint)
tokenizer.model_max_len=512

# model
#config = RobertaConfig.from_pretrained(checkpoint, output_attentions=True, output_hidden_states=True)
config = RobertaConfig.from_pretrained(checkpoint)
config.output_hidden_states = True
#config.output_attentions= True
model = RobertaForSequenceClassification.from_pretrained(checkpoint, config=config)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at /work3/s174498/finetuning-sentiment-model-test-head3/checkpoint-500 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [32]:
# Prepare the text inputs for the model
def preprocess_function(examples):
    return tokenizer(examples["sentence"], truncation=True, max_length=512)

In [33]:
tokenized_test = test_dataset.map(preprocess_function, batched=True)

  0%|          | 0/2 [00:00<?, ?ba/s]

In [34]:
tokenized_test

Dataset({
    features: ['idx', 'sentence', 'label', 'input_ids', 'attention_mask'],
    num_rows: 1821
})

In [35]:
trainer = Trainer(
    model=model,                        
    tokenizer=tokenizer
)

In [36]:
# Predicting with model
predictions = trainer.predict(tokenized_test)


The following columns in the test set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: sentence, idx. If sentence, idx are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 1821
  Batch size = 8


In [74]:
pred_dataset_test = list(np.argmax(predictions.predictions[0], axis=-1))
true_dataset_test = predictions.label_ids

In [75]:
# compute accuracy
accuracy_metric = load_metric("accuracy")
accuracy_metric.compute(predictions=pred_dataset_test, references=true_dataset_test)

{'accuracy': 0.9341021416803954}